In [ ]:
import pandas as pd
import zipfile
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
import cv2
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_curve, auc

In [ ]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            images.append(img_gray)
    return images

cleared the ouput cells in which images are to be shown because the file was unable to download

In [ ]:
images_daisy = load_images_from_folder(r'C:\Users\Nimra Amer\Downloads\archive\flowers\daisy')
for img in images_daisy:
    plt.imshow(img)
    plt.show()

In [ ]:
images_dandelion = load_images_from_folder(r'C:\Users\Nimra Amer\Downloads\archive\flowers\dandelion')
for img in images_dandelion:
    plt.imshow(img)
    plt.show()

In [ ]:
images_rose = load_images_from_folder(r'C:\Users\Nimra Amer\Downloads\archive\flowers\rose')
for img in images_rose:
    plt.imshow(img)
    plt.show()

In [ ]:
images_sunflower = load_images_from_folder(r'C:\Users\Nimra Amer\Downloads\archive\flowers\sunflower')
for img in images_sunflower:
    plt.imshow(img)
    plt.show()

In [ ]:
images_tulip = load_images_from_folder(r'C:\Users\Nimra Amer\Downloads\archive\flowers\tulip')
for img in images_tulip:
    plt.imshow(img)
    plt.show()

In [ ]:
images = images_daisy + images_dandelion + images_rose + images_sunflower + images_tulip

Scaling b/w 0 and 1

In [ ]:
images_scaled = []
labels = []

for img in images:
    images_scaled.append(img.astype(np.float32) / 255.0)


labels = (['daisy'] * len(images_daisy) +
          ['dandelion'] * len(images_dandelion) +
          ['rose'] * len(images_rose) +
          ['sunflower'] * len(images_sunflower) +
          ['tulip'] * len(images_tulip))

labels = np.array(labels)

images_scaled

[array([[0.5294118 , 0.5411765 , 0.5568628 , ..., 0.6       , 0.6117647 ,
         0.5803922 ],
        [0.5254902 , 0.5372549 , 0.5529412 , ..., 0.6       , 0.6117647 ,
         0.5803922 ],
        [0.52156866, 0.53333336, 0.5529412 , ..., 0.6       , 0.60784316,
         0.57254905],
        ...,
        [0.1764706 , 0.17254902, 0.17254902, ..., 0.49803922, 0.49803922,
         0.49411765],
        [0.17254902, 0.17254902, 0.17254902, ..., 0.49803922, 0.49803922,
         0.49803922],
        [0.17254902, 0.17254902, 0.17254902, ..., 0.5058824 , 0.5058824 ,
         0.5058824 ]], dtype=float32),
 array([[0.85882354, 0.8352941 , 0.8156863 , ..., 0.03921569, 0.02745098,
         0.03921569],
        [0.8352941 , 0.8666667 , 0.85882354, ..., 0.03137255, 0.03137255,
         0.02745098],
        [0.88235295, 0.85490197, 0.8666667 , ..., 0.03137255, 0.04313726,
         0.02352941],
        ...,
        [0.8901961 , 0.8745098 , 0.85882354, ..., 0.6745098 , 0.67058825,
         0.62352943

In [ ]:
target_size = (224, 224)
resized_images = []

for img in images_scaled:
    resized_img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)
    resized_images.append(resized_img)

Splitting Dataset in Trainning and Validation Sets

In [ ]:
X = np.array(resized_images)
Y = labels

X_train, X_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.2, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

Initialize the weights and biases of the MLP using random values.

In [ ]:
def weights_biases(layers):
    weights_bias = {}
    num_layers = len(layers) - 1  # Number of layers (excluding input layer)

    for num in range(1, num_layers + 1):
        current_layer_size = layers[num]
        previous_layer_size = layers[num - 1]

        # Initialize weights and biases for the current layer
        weights_bias['W' + str(num)] = np.random.randn(current_layer_size, previous_layer_size) * 0.01
        weights_bias['b' + str(num)] = np.zeros((current_layer_size, 1))

    return weights_bias

Implement the forward pass of the MLP, where you compute the output of each neuron in
each layer.

In [ ]:
#Initializing layers
num_examples, height, width = X_train.shape
input_layer = height * width
hidden_layers = [128 , 128]
output_layer = len(np.unique(y_train))

In [ ]:
#Activation Functions
def sigmoid(x):  #for hidden Layers
    return (1/(1+np.exp(-x)))

def softmax(x): #for output layer
    exp_x = np.exp(x - np.max(x, axis=0))
    return exp_x / np.sum(exp_x, axis=0)

def forward_pass_MLP(input, weights_bias):
    x = input.reshape(input.shape[0], -1).T
    num_layers = len(weights_bias) // 2
    answers = []

    for num in range(1 , num_layers+1):
        weights = weights_bias['W' + str(num)]
        bias = weights_bias['b' + str(num)]
        equation = np.dot(weights , x) + bias #wx + b
        result = sigmoid(equation)
        answers.append((equation , result))
        x = result

    #for output
    Weights_output = weights_bias['W' + str(num_layers)]
    Bias_output = weights_bias['b' + str(num_layers)]
    equation_output = np.dot(Weights_output.T , result) + Bias_output
    result_output = softmax(equation_output)

    answers.append(result_output)

    return result_output , answers

In [ ]:
layers = [input_layer] + hidden_layers + [output_layer]
parameters = weights_biases(layers)

train_predict , ans_train_predict = forward_pass_MLP(X_train, parameters)
val_predict , ans_val_predict = forward_pass_MLP(X_val, parameters)

Implement the backward pass of the MLP, where you compute the gradient of the loss
function with respect to each weight and bias in the network.

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=0))
    return exp_x / np.sum(exp_x, axis=0, keepdims=True)


def backward_pass_MLP(X, Y, weights_bias, answers):
    num_layers = len(weights_bias) // 2
    num = X.shape[1]

    gradients = {}

    Answer = answers[-1] - Y
    result = Answer * softmax(answers[-1])

    previous = answers[-2]
    gradients['dW' + str(num_layers)] = np.dot(result, A_prev.T) / num
    gradients['db' + str(num_layers)] = np.sum(result, axis=1, keepdims=True) / num

    curr = result

    for l in reversed(range(1, num_layers)):
        W_current = weights_bias['W' + str(l)]
        Z_prev, A_prev = answers[l - 1]

        dZ = np.dot(W_current.T, dA) * sigmoid(Z_prev) * (1 - sigmoid(Z_prev))
        gradients['dW' + str(l)] = np.dot(dZ, A_prev.T) / num
        gradients['db' + str(l)] = np.sum(dZ, axis=1, keepdims=True) / num

        dA = dZ

    # Input layer gradients
    W_first = weights_bias['W1']
    Z_first, _ = answers[0]

    dZ_first = np.dot(W_first.T, dA) * sigmoid(Z_first) * (1 - sigmoid(Z_first))
    gradients['dW1'] = np.dot(dZ_first, X.T) / num
    gradients['db1'] = np.sum(dZ_first, axis=1, keepdims=True) / num

    return gradients

gradients_train = backward_pass_MLP(X, Y, weights_biases, ans_train_predict)
gradients_val = backward_pass_MLP(X, Y, weights_biases, ans_val_predict)

Update the weights and biases using stochastic gradient descent optimization algorithm
with a learning rate of 0.01.

In [ ]:
alpha = 0.01
def update_parameters(parameters, gradients, alpha):
    num_layers = len(parameters) // 2
    for l in range(1, num_layers + 1):
        parameters['W' + str(l)] -= alpha * gradients['dW' + str(l)]
        parameters['b' + str(l)] -= alpha * gradients['db' + str(l)]

    return parameters


Train the MLP for 10 epochs, and monitor the accuracy on the validation set after each
epoch.

In [ ]:
for epoch in range(10):
    output_train, _ = forward_pass_MLP(X_train, parameters)
    predictions_train = np.argmax(output_train, axis=0)

    acc_train = accuracy_score(y_train, predictions_train)
    output_val, _ = forward_pass_MLP(X_val, parameters)
    predictions_val = np.argmax(output_val, axis=0)

    acc_val = accuracy_score(y_val, predictions_val)
    print("Epoch " , epoch + 1 , ": Trainning Accuracy = " , acc_train , " Validation Accuracy : " , acc_val)
    parameters = update_parameters(parameters , gradients_train , alpha)

Evaluate the final accuracy of the trained MLP on the test set, and calculate the
classification report, which includes precision, recall, and F1-score for each class.

In [ ]:
output = forward_pass_MLP(X_test, parameters)
predictions = np.argmax(output, axis=0)
acc = accuracy_score(y_test, predictions)
print("Accuracy : " , acc)

label_encoder = LabelEncoder()
y_test_encoded = label_encoder.fit_transform(y_test)
predictions_test_encoded = label_encoder.transform(predictions)

target_names = label_encoder.classes_
report = classification_report(y_test_encoded, predictions_test_encoded, target_names=target_names)
print("Classification Report:")
print(report)

Plot the ROC curve for the classifier, and calculate the area under the curve (AUC).

In [ ]:
y_pred_probs = output[1]
fpr, tpr, thresholds = roc_curve(y_test_encoded, y_pred_probs)

roc_auc = auc(fpr, tpr)
print("AUC:", roc_auc)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='b', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()